# Predobdelava podatkov

## Okolje

Vzpostavitev okolja

In [1]:
!pip install tweet-preprocessor
!pip install classla

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 275 kB 32.5 MB/s 
     |████████████████████████████████| 76 kB 5.0 MB/s 
     |████████████████████████████████| 262 kB 74.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 1.1 MB 45.8 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:39tcmalloc: large alloc 1147494400 bytes == 0x39bb8000 @  0x7f1c634d3615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 15 kB/s 
     |████████████████████████████████| 262 kB 7

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import json
import classla
import re
import time

classla.download('sl')

import preprocessor as tpre

from google.colab import drive
drive.mount('/content/drive/')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
2022-08-09 12:03:55 INFO: Downloading these customized packages for language: sl (Slovenian)...
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |
| pretrain  | standard |

2022-08-09 12:06:48 INFO: Finished downloading models and saved to /root/classla_resources.


Mounted at /content/drive/


In [3]:
# Setting constants

LOCAL = False

google_data_dir = "/content/drive/MyDrive/Diploma/Data"
local_data_dir = "/data"

root_dir = ""
if LOCAL:
    root_dir = local_data_dir
else:
    root_dir = google_data_dir

## Funkcije in razredi

In [16]:
def preprocess_tweets(preprocess_pipeline, tweets, tweet_stop_words=[], tweet_upos=[], min_words=4, verbose=False, debug=False):

  stop_words = stopwords.words('slovene')
  stop_words.extend(tweet_stop_words)

  # Remove stopwords
  stop_words = list(set(stop_words))

  data = []
  start_time = 0

  for index, tweet in enumerate(tweets):

    if index%1000 == 0 and verbose:
      # Import time
      print(f'-- Progress: {index}/{len(tweets)}')
      if index >= 1000:
        print(f'-- Time elapsed: {time.time() - start_time}s')
        print(f'-- Tweets preprocessed: {len(data)}')
      start_time = time.time()

    # Take attributes
    tweet_full_text = tweet['full_text']

    # Skip if retweet
    if tweet_full_text.startswith("RT"):
      continue

    tweet_id = tweet['id']
    #tweet_hashtags = tweet['entities']['hashtags']
    #tweet_mentions = tweet['entities']['mentions']
    tweet_hashtags = tweet['hashtags']
    tweet_mentions = tweet['mentions']
    tweet_created_at = tweet['created_at']

    tweet_user_name = tweet['user']['name']
    tweet_user_screen_name = tweet['user']['screen_name']
    tweet_user_description = clean_tweet_text(tweet['user']['description']).lower()

    # Remove hashtags, mentions, links, emojis and others
    tweet_full_text = clean_tweet_text(tweet_full_text)

    if debug:
      print(f'Raw text:\n{tweet_full_text}')

    # Preprocess with preprocessing pipeline
    tweet_lemma_text = preprocess_pipeline(tweet_full_text)

    tweet_processed_lemmas = []
    for sentence in tweet_lemma_text.sentences:

      sentence_words = []

      for i, word in enumerate(sentence.words):
 
        # If the tweet is a retweet
        if word.lemma.lower() == 'rt' and i == 0:
          break

        if debug:
          print(f'Lemma: {word.lemma.lower()} -------------- Upos: {word.upos}')

        # Not punctuation and not number + clean stopwords
        if word.upos not in tweet_upos and word.lemma.lower() not in stop_words:
          sentence_words.append(word.lemma.lower())
        elif debug:
          print(f'Discarded word: {word.lemma}')

      tweet_processed_lemmas.extend(sentence_words)
    
    if debug:
      print(f'Preprocessed lemmas:\n{tweet_processed_lemmas}')

    # If less than n words
    if len(tweet_processed_lemmas) > min_words:
      tweet_data = {
          "id": tweet_id,
          "created_at": tweet_created_at,
          "raw_text": tweet_full_text,
          "lemma_text": ' '.join(tweet_processed_lemmas),
          "hashtags": tweet_hashtags,
          "mentions": tweet_mentions,
          "user": {
              "name": tweet_user_name,
              "screen_name": tweet_user_screen_name,
              "description": tweet_user_description
          }
      }

      # Append all the data
      data.append(tweet_data)
    
  return data

def clean_tweet_text(tweet_text):
  tweet_text = tpre.clean(tweet_text)
  tweet_text = re.sub("&gt;|&lt;|&amp;", "", tweet_text)
  tweet_text = remove_emojis(tweet_text)
  return tweet_text

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def load_tweets(file_name):
  
  # Load data
  data = []

  with open(file_name, 'r', encoding='utf8') as sample_data:
    data = json.load(sample_data)

  return data

def save_tweets(data, dir, file_name):
  with open(f'{dir}/{file_name}.json', 'w+', encoding='utf8') as outdata:
    json.dump(data, outdata, ensure_ascii=False)

def load_and_preprocess(cpipeline, data_dir, only_load, tweet_stop_words=[], tweet_upos=[], min_words=4, verbose=False, debug=False):
  d = []
  if only_load:
    d = load_tweets(data_dir)
  else:
    d = preprocess_tweets(cpipeline, load_tweets(data_dir), tweet_stop_words=tweet_stop_words, tweet_upos=tweet_upos, min_words=min_words, verbose=verbose, debug=debug)
  return d

In [14]:
# Setting configuration

# Path to unpreprocessed tweet data

YEAR = 2021
EPOCH = 2
BATCH = 1

tweet_data_path = f'unpreprocess/{YEAR}-{EPOCH}/{YEAR}_{EPOCH}_{BATCH}.json'

# Path to save processed tweet data
SAVING = True
tweet_save_path = f'preprocess/{YEAR}-{EPOCH}'

# Preprocessing configuration
preprocess_config = {
    'only_load': False,
    'min_words': 4,
    'verbose': True,
    'debug': False,
    'tweet_upos': ['PUNCT', 'NUM', 'SYM', 'CCONJ', 'INTJ'],
    'tweet_stop_words': ['http', 'https', 'rt', 'oz']
}

# Classla configuration
classla_conf = {
  #'processors': 'tokenize, lemma',
  'lang': 'sl',
  'pos_lemma_pretag' : True,
  'use_gpu': True
}


In [6]:
# Loading preprocessors

# Classla preprocessor
classla_pipeline = classla.Pipeline(**classla_conf)

# Tweet preprocessor
tpre.set_options(tpre.OPT.URL, tpre.OPT.MENTION, tpre.OPT.HASHTAG)

2022-08-09 12:07:13 INFO: Loading these models for language: sl (Slovenian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |

2022-08-09 12:07:13 INFO: Use device: gpu
2022-08-09 12:07:13 INFO: Loading: tokenize
2022-08-09 12:07:13 INFO: Loading: pos
2022-08-09 12:07:33 INFO: Loading: lemma
2022-08-09 12:07:43 INFO: Loading: depparse
2022-08-09 12:07:44 INFO: Loading: ner
2022-08-09 12:07:44 INFO: Done loading processors!


In [17]:
# MAIN CODE

# Preprocess tweet data
preprocessed_tweet_data = load_and_preprocess(classla_pipeline, f'{root_dir}/{tweet_data_path}', **preprocess_config)

if SAVING:
    print(f'Saving preprocessed tweets to {root_dir}/{tweet_save_path}...')
    save_tweets(preprocessed_tweet_data, f'{root_dir}/{tweet_save_path}', f'{YEAR}_{EPOCH}_{BATCH}')

# Print summary
print(f'- Batch summary:')
print(f'-- Batch length: {len(preprocessed_tweet_data)}')

-- Progress: 0/10000
-- Progress: 1000/10000
-- Time elapsed: 38.14611029624939s
-- Tweets preprocessed: 274
-- Progress: 2000/10000
-- Time elapsed: 36.95746469497681s
-- Tweets preprocessed: 568
-- Progress: 3000/10000
-- Time elapsed: 47.25840950012207s
-- Tweets preprocessed: 931
-- Progress: 4000/10000
-- Time elapsed: 53.898706912994385s
-- Tweets preprocessed: 1324
-- Progress: 5000/10000
-- Time elapsed: 49.66877889633179s
-- Tweets preprocessed: 1727
-- Progress: 6000/10000
-- Time elapsed: 47.81388068199158s
-- Tweets preprocessed: 2106
-- Progress: 7000/10000
-- Time elapsed: 44.154149770736694s
-- Tweets preprocessed: 2445
-- Progress: 8000/10000
-- Time elapsed: 41.39900350570679s
-- Tweets preprocessed: 2766
-- Progress: 9000/10000
-- Time elapsed: 49.98219013214111s
-- Tweets preprocessed: 3157
Saving preprocessed tweets to /content/drive/MyDrive/Diploma/Data/preprocess/2021-2...
- Batch summary:
-- Batch length: 3542
